# Code 1 - SQL Agent 
- Open API
- Langchain
- sqlite database

### Referances
- sqlite (download portable package) https://sqlitebrowser.org/dl/

## Step 1 - Connect to Database

### Import Packages

In [1]:
from sqlite3 import connect
from langchain_community.utilities.sql_database import SQLDatabase

### Create Connection URL

In [2]:
db = SQLDatabase.from_uri("sqlite:///chinook.db", sample_rows_in_table_info = 3)

### Check Databases 
- if the database is blank, please use sqlite browser to populate the same

In [3]:
print(len(db.get_usable_table_names()), db.get_usable_table_names())

11 ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


In [4]:
print(db.table_info)


CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Empl

### Connect to sqlite database

In [5]:
con = connect("chinook.db")

### Check Sample Query

In [6]:
cur = con.execute("Select Count(Distinct(AlbumId)) from Album;")

In [7]:
print(cur.fetchall())

[(347,)]


## Step 2 - Setup Langchain SQL Chain

### Load Environment Variables 
- Mostly API Keys

In [8]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
API_KEY = os.getenv("OPENAI_API_KEY")

### Load Packages

In [9]:
from langchain.callbacks import get_openai_callback
from langchain_openai import ChatOpenAI

### Create an Instance of LLM with configuration

In [10]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key = API_KEY)

### Check LLM 

In [11]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000248ED3E1F50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000248EEADED10>, root_client=<openai.OpenAI object at 0x00000248EE8C26D0>, root_async_client=<openai.AsyncOpenAI object at 0x00000248EEAD2AD0>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

## Step 3 - Create Chain

### Import Packages

In [12]:
from langchain_experimental.sql.base import SQLDatabaseChain

### Setup SQL DB Chain

In [13]:
sql_db_chain = SQLDatabaseChain.from_llm(llm = llm, db = db, verbose = True)

### Run and Print Chain Output

In [14]:
result = sql_db_chain.invoke("Which country's customers spent the most?")
print(result)



> Entering new SQLDatabaseChain chain...
Which country's customers spent the most?
SQLQuery:SELECT c."Country", SUM(i."Total") AS total_spent
FROM "Customer" c
JOIN "Invoice" i ON c."CustomerId" = i."CustomerId"
GROUP BY c."Country"
ORDER BY total_spent DESC
LIMIT 1;
SQLResult: [('USA', 523.06)]
Answer:The country whose customers spent the most is USA, with a total amount spent of $523.06.
> Finished chain.
{'query': "Which country's customers spent the most?", 'result': 'The country whose customers spent the most is USA, with a total amount spent of $523.06.'}


### Run and Print Output with Callback
- Get # Tokens used
- Get # Requests
- Get # Completion Tokens

In [15]:
with get_openai_callback() as cb:
    result = sql_db_chain.invoke("Which country's customers spent the most?")
print("LLM Result: ", result['result'])
print(cb)



> Entering new SQLDatabaseChain chain...
Which country's customers spent the most?
SQLQuery:SELECT c."Country", SUM(i."Total") AS total_spent
FROM "Customer" c
JOIN "Invoice" i ON c."CustomerId" = i."CustomerId"
GROUP BY c."Country"
ORDER BY total_spent DESC
LIMIT 1;
SQLResult: [('USA', 523.06)]
Answer:The country whose customers spent the most is USA, with a total amount spent of $523.06.
> Finished chain.
LLM Result:  The country whose customers spent the most is USA, with a total amount spent of $523.06.
Tokens Used: 4880
	Prompt Tokens: 4806
	Completion Tokens: 74
Successful Requests: 2
Total Cost (USD): $0.0025139999999999997


In [16]:
with get_openai_callback() as cb:
    result = sql_db_chain.invoke("List the total sales per country. Which country's customers spent the most?")
print("LLM Result: ", result['result'])
print(cb)



> Entering new SQLDatabaseChain chain...
List the total sales per country. Which country's customers spent the most?
SQLQuery:SELECT c."Country", SUM(i."Total") AS "TotalSales"
FROM "Customer" c
JOIN "Invoice" i ON c."CustomerId" = i."CustomerId"
GROUP BY c."Country"
ORDER BY "TotalSales" DESC
LIMIT 1;
SQLResult: [('USA', 523.06)]
Answer:The country whose customers spent the most is USA with a total sales of $523.06.
> Finished chain.
LLM Result:  The country whose customers spent the most is USA with a total sales of $523.06.
Tokens Used: 4894
	Prompt Tokens: 4821
	Completion Tokens: 73
Successful Requests: 2
Total Cost (USD): $0.0025199999999999997


### Check Template

In [17]:
print(sql_db_chain.llm_chain.prompt.template)

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: 

# END - Next Topic - SQL Agents